In [55]:
import pandas as pd
import  numpy as np
import os

In [ ]:
base_path=r"C:\Users\layan\OneDrive\Desktop\UCI_HAR_Dataset\UCI_HAR_Dataset"
def load_signals_correctly(base_path, dataset_type='train'):
    """Load signals in the correct UCI HAR format"""
    signals_path = os.path.join(base_path, dataset_type, 'Inertial_Signals')
    
    
    signal_files = [
        f"total_acc_x_{dataset_type}.txt",
        f"total_acc_y_{dataset_type}.txt", 
        f"total_acc_z_{dataset_type}.txt",
        f"body_gyro_x_{dataset_type}.txt",
        f"body_gyro_y_{dataset_type}.txt",
        f"body_gyro_z_{dataset_type}.txt"
    ]
    
    signals = {}
    for f in signal_files:
        key = f.split('_')[0] + '_' + f.split('_')[1]  
        if key not in signals:
            signals[key] = []
        signal_data = np.loadtxt(os.path.join(signals_path, f))
        signals[key].append(signal_data)
    
    
    for key in signals:
        signals[key] = np.stack(signals[key], axis=-1)  
    
    return signals


train_signals = load_signals_correctly(base_path, 'train')
test_signals = load_signals_correctly(base_path, 'test')


total_acc_train_windows = train_signals['total_acc']  
gyro_train_windows = train_signals['body_gyro']       
total_acc_test_windows = test_signals['total_acc']    
gyro_test_windows = test_signals['body_gyro']        

print("Correct train acc windows shape:", total_acc_train_windows.shape)
print("Correct train gyro windows shape:", gyro_train_windows.shape)
print("Correct test acc windows shape:", total_acc_test_windows.shape)
print("Correct test gyro windows shape:", gyro_test_windows.shape)

Correct train acc windows shape: (7352, 128, 3)
Correct train gyro windows shape: (7352, 128, 3)
Correct test acc windows shape: (2947, 128, 3)
Correct test gyro windows shape: (2947, 128, 3)


In [57]:
def load_labels_for_your_structure(base_path):
  
    y_train_path = os.path.join(base_path, 'train', 'y_train.txt')
    y_test_path = os.path.join(base_path, 'test', 'y_test.txt')
    
    print("=== LOADING LABELS FOR YOUR STRUCTURE ===")
    print(f"Training labels path: {y_train_path}")
    print(f"Test labels path: {y_test_path}")
    
    def load_single_file(file_path, label_type):
        if os.path.exists(file_path):
            try:
                y_data = np.loadtxt(file_path).astype(int)
                print(f"✓ Loaded {label_type} labels: {y_data.shape}")
                return y_data
            except Exception as e:
                print(f"✗ Error loading {label_type}: {e}")
                return None
        else:
            print(f"✗ File not found: {file_path}")
            return None
    
    y_train = load_single_file(y_train_path, 'training')
    y_test = load_single_file(y_test_path, 'test')
    
    return y_train, y_test

base_path = r"C:\Users\layan\OneDrive\Desktop\UCI_HAR_Dataset\UCI_HAR_Dataset"
y_train, y_test = load_labels_for_your_structure(base_path)

=== LOADING LABELS FOR YOUR STRUCTURE ===
Training labels path: C:\Users\layan\OneDrive\Desktop\UCI_HAR_Dataset\UCI_HAR_Dataset\train\y_train.txt
Test labels path: C:\Users\layan\OneDrive\Desktop\UCI_HAR_Dataset\UCI_HAR_Dataset\test\y_test.txt
✓ Loaded training labels: (7352,)
✓ Loaded test labels: (2947,)


In [ ]:
def load_activity_names_for_your_structure(base_path):
    """
    Load activity_labels.txt from your specific structure
    """
    activity_path = os.path.join(base_path, 'activity_labels.txt')
    
    print(f"\nLoading activity names from: {activity_path}")
    print(f"File exists: {os.path.exists(activity_path)}")
    
    if os.path.exists(activity_path):
        try:
            
            activity_df = pd.read_csv(activity_path, sep=' ', header=None, 
                                     names=['activity_id', 'activity_name'])
            activity_mapping = dict(zip(activity_df['activity_id'], activity_df['activity_name']))
            
            print("✓ Activity mapping loaded:")
            for activity_id, activity_name in activity_mapping.items():
                print(f"  {activity_id}: {activity_name}")
                
            return activity_mapping
        except Exception as e:
            print(f"✗ Error loading activity names: {e}")
            return None
    else:
        print(f"✗ Activity labels file not found")
        return None


activity_mapping = load_activity_names_for_your_structure(base_path)


Loading activity names from: C:\Users\layan\OneDrive\Desktop\UCI_HAR_Dataset\UCI_HAR_Dataset\activity_labels.txt
File exists: True
✓ Activity mapping loaded:
  1: WALKING
  2: WALKING_UPSTAIRS
  3: WALKING_DOWNSTAIRS
  4: SITTING
  5: STANDING
  6: LAYING


In [ ]:
features_train = np.nan_to_num(features_train, nan=0.0)
features_test = np.nan_to_num(features_test, nan=0.0)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, random_state=42, verbose=1)
clf.fit(features_train, y_train)

print("🎉 Training completed successfully! 🎉")

train_accuracy = clf.score(features_train, y_train)
test_accuracy = clf.score(features_test, y_test)

print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

y_pred = clf.predict(features_test)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    7.5s


🎉 Training completed successfully! 🎉
Train Accuracy: 1.0000
Test Accuracy: 0.8996


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


In [60]:


from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import time

print("Scaling features for SVM...")
scaler = StandardScaler()
features_train_scaled = scaler.fit_transform(features_train)
features_test_scaled = scaler.transform(features_test)

kernels = ['linear', 'rbf', 'poly']
svm_results = {}

for kernel in kernels:
    print(f"\nTraining SVM with {kernel} kernel...")
    start_time = time.time()
    
    svm_clf = SVC(kernel=kernel, random_state=42, verbose=1)
    svm_clf.fit(features_train_scaled, y_train)
    
    y_pred_train = svm_clf.predict(features_train_scaled)
    y_pred_test = svm_clf.predict(features_test_scaled)
    
    train_accuracy = accuracy_score(y_train, y_pred_train)
    test_accuracy = accuracy_score(y_test, y_pred_test)
    
    training_time = time.time() - start_time
    
    svm_results[kernel] = {
        'train_accuracy': train_accuracy,
        'test_accuracy': test_accuracy,
        'training_time': training_time,
        'model': svm_clf
    }
    
    print(f"SVM ({kernel}) - Train: {train_accuracy:.4f}, Test: {test_accuracy:.4f}, Time: {training_time:.2f}s")

print("\n" + "="*50)
print("COMPARISON OF MODELS")
print("="*50)

print(f"Random Forest:")
print(f"  Train Accuracy: {1.0000:.4f}")
print(f"  Test Accuracy:  {0.8996:.4f}")

for kernel, results in svm_results.items():
    print(f"\nSVM ({kernel}):")
    print(f"  Train Accuracy: {results['train_accuracy']:.4f}")
    print(f"  Test Accuracy:  {results['test_accuracy']:.4f}")
    print(f"  Training Time:  {results['training_time']:.2f}s")

best_svm_kernel = max(svm_results.items(), key=lambda x: x[1]['test_accuracy'])
print(f"\nBest SVM kernel: {best_svm_kernel[0]} with test accuracy: {best_svm_kernel[1]['test_accuracy']:.4f}")

best_svm = best_svm_kernel[1]['model']
y_pred_best_svm = best_svm.predict(features_test_scaled)

print("\n" + "="*50)
print("DETAILED PERFORMANCE - BEST SVM")
print("="*50)
print(classification_report(y_test, y_pred_best_svm, 
                          target_names=['Walking', 'Walking Upstairs', 'Walking Downstairs', 
                                       'Sitting', 'Standing', 'Laying']))

Scaling features for SVM...

Training SVM with linear kernel...
[LibSVM]SVM (linear) - Train: 0.9962, Test: 0.9596, Time: 2.47s

Training SVM with rbf kernel...
[LibSVM]SVM (rbf) - Train: 0.9901, Test: 0.9477, Time: 12.91s

Training SVM with poly kernel...
[LibSVM]SVM (poly) - Train: 0.9819, Test: 0.9186, Time: 9.70s

COMPARISON OF MODELS
Random Forest:
  Train Accuracy: 1.0000
  Test Accuracy:  0.8996

SVM (linear):
  Train Accuracy: 0.9962
  Test Accuracy:  0.9596
  Training Time:  2.47s

SVM (rbf):
  Train Accuracy: 0.9901
  Test Accuracy:  0.9477
  Training Time:  12.91s

SVM (poly):
  Train Accuracy: 0.9819
  Test Accuracy:  0.9186
  Training Time:  9.70s

Best SVM kernel: linear with test accuracy: 0.9596

DETAILED PERFORMANCE - BEST SVM
                    precision    recall  f1-score   support

           Walking       0.97      0.98      0.98       496
  Walking Upstairs       0.97      0.97      0.97       471
Walking Downstairs       0.98      0.97      0.97       420
     

In [ ]:
import os

print(os.getcwd())  
print(os.listdir(r"C:\Users\layan\OneDrive\Desktop"))  


c:\Users\layan\OneDrive\Desktop\motionsense
['1683385379_g.jpg', '23-Replica-Jersey-Youth.webp', '230621042149-01-cristiano-ronaldo-euro-200-apps-062023-restricted.jpg', '51MBon-HIPL._AC_UY1000_.jpg', '57498860xlarge.jpg', '@gjumun _ Linktree.html', '@gjumun _ Linktree_files', 'Anki.lnk', 'app.py', 'assignment3.cpp', 'Autodesk Fusion.lnk', 'Blender 4.2.lnk', 'bst.c', 'ButtonSwap', 'c', 'calc', 'calculator', 'CapCut.lnk', 'circuit.circ', 'circular linked list.c', 'CppApplication_1', 'CppApplication_2', 'CS', 'cs even and odd arrays.c', 'cs for float practice.txt', 'cs frequency of each element practice.c', 'cs funtcions.txt', 'cs if statement practice.c', 'cs if ststement practice.c', 'cs merging arrays.c', 'cs perfect number practice.txt', 'cs practic 2.c', 'cs practice insert a value in an array.c', 'cs practice1.c', 'cs quiz.txt', 'cs root.c', 'cs sorting arrays.c', 'cs1', 'CS214 Lecture 7.pdf', 'database final', 'Delegate Applications (Responses).xlsx', 'description.docx', 'desktop.

In [ ]:


print("\n" + "="*50)
print("SAVING COMPLETE PREDICTOR FOR DEPLOYMENT")
print("="*50)


from har_utils import HARPredictor

predictor = HARPredictor(best_svm, scaler, activity_mapping)

joblib.dump(predictor, 'har_predictor_complete.pkl')
print("✅ Complete predictor saved as 'har_predictor_complete.pkl'")


print("\n🧪 Testing the saved predictor...")
test_predictions = predictor.predict(total_acc_test_windows[:5], gyro_test_windows[:5])
print("Predictions:", test_predictions)


actual_activities = [activity_mapping[label] for label in y_test[:5]]
print("Actual:     ", actual_activities)

print("\n🎉 Deployment predictor is ready! Use har_predictor_complete.pkl with har_utils.py")


SAVING COMPLETE PREDICTOR FOR DEPLOYMENT
✅ Complete predictor saved as 'har_predictor_complete.pkl'

🧪 Testing the saved predictor...
Predictions: ['STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING']
Actual:      ['STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING']

🎉 Deployment predictor is ready! Use har_predictor_complete.pkl with har_utils.py
